In [1]:
import pandas as pd
import numpy as np
import requests
import json
import time

api_key = 'RGAPI-6be3f8a6-6833-4e82-a374-00aba6f803fd'

In [2]:
def get_match_infos_from_id(matchId, api_key):
    url = f'https://americas.api.riotgames.com/lol/match/v5/matches/BR1_{matchId}'
    header = {'api_key':api_key}
    r = requests.get(url, params=header)
    return r.json()

def get_id_from_url(matchUrl):
    match_details = matchUrl.split('#match-details/')[1]
    return match_details.split('/')[1]

def get_match_infos_from_url(matchUrl, api_key):
    matchId = get_id_from_url(matchUrl)
    return get_match_infos_from_id(matchId, api_key)

def get_list_of_ids_from_ulrs(list_of_urls):
    list_of_ids = []
    for url in list_of_urls:
        list_of_ids.append(get_id_from_url(url))
    return np.unique(list_of_ids)

def get_multiple_infos_from_urls(list_of_urls, api_key):
    list_of_infos = []
    for index, url in enumerate(np.unique(list_of_urls)):
        list_of_infos.append(get_match_infos_from_url(url, api_key))
        if index % 19 == 0:
            time.sleep(1)
    return list_of_infos

def info_to_players_dataframe(match_infos):
    players = pd.DataFrame(match_infos['info']['participants'])
    players['gameId']             = match_infos['info']['gameId']
    players['gameDuration']       = match_infos['info']['gameDuration']
    players['gameStartTimestamp'] = match_infos['info']['gameStartTimestamp']

    return players

def csv_urls_to_participants_df(path_to_csv, api_key):
    full_match_list = list(pd.read_csv(path_to_csv).match_url)

    list_of_infos  = get_multiple_infos_from_urls(full_match_list, api_key)
    matches_as_dfs = [info_to_players_dataframe(info) for info in list_of_infos]
    full_matches   = pd.concat(matches_as_dfs)

    return full_matches

In [54]:
df_full = csv_urls_to_participants_df('../list_of_matches.csv', api_key)

In [56]:
for col in df_full.columns:
    print(col)

assists
baronKills
bountyLevel
champExperience
champLevel
championId
championName
championTransform
consumablesPurchased
damageDealtToBuildings
damageDealtToObjectives
damageDealtToTurrets
damageSelfMitigated
deaths
detectorWardsPlaced
doubleKills
dragonKills
firstBloodAssist
firstBloodKill
firstTowerAssist
firstTowerKill
gameEndedInEarlySurrender
gameEndedInSurrender
goldEarned
goldSpent
individualPosition
inhibitorKills
inhibitorsLost
item0
item1
item2
item3
item4
item5
item6
itemsPurchased
killingSprees
kills
lane
largestCriticalStrike
largestKillingSpree
largestMultiKill
longestTimeSpentLiving
magicDamageDealt
magicDamageDealtToChampions
magicDamageTaken
neutralMinionsKilled
nexusKills
nexusLost
objectivesStolen
objectivesStolenAssists
participantId
pentaKills
perks
physicalDamageDealt
physicalDamageDealtToChampions
physicalDamageTaken
profileIcon
puuid
quadraKills
riotIdName
riotIdTagline
role
sightWardsBoughtInGame
spell1Casts
spell2Casts
spell3Casts
spell4Casts
summoner1Casts
su

In [64]:
cols_analise = ['puuid', 'summonerName','win','detectorWardsPlaced', 'gameId']
df_full.loc[:, cols_analise]

,puuid,summonerName,win,detectorWardsPlaced,gameId
0,q2vqZN9CCembJV_C2aIpaT8qSGmyW-H4cA92g7y4mWH4QO...,Ohto,True,4,2254123794
1,Xl9wzyX9Hnw6v3lFKpSvISadkxJO2KyS4AOs8lxirULLTL...,Axium,True,1,2254123794
2,ouK21DUGdQa1S-38CPYKPsZVQzd5o72Uk4gn_FWzK7Biku...,kindle eyes,True,4,2254123794
3,dSrm4Xq9PEcPGX2mFJ5NfDe0l4IlttTLGvk42VBtS7bNkl...,Nabruxa,True,1,2254123794
4,w42D9MWwWYgP6WBnePIVvA3odKnIYB-LvmjH2pvN6MKUln...,LadyKell,True,2,2254123794
5,Vctog9y_7tyGkSVwiSrgeLmpIs5_5r_BEetZ8888mL3oRf...,Jpguima,False,0,2254123794
6,U-l5FEkys6K6fVlYDHCLT7kzi3A2x015vjpGCC80Vi5SQ5...,EduTenorio,False,1,2254123794
7,EceUuasZqIXn5mPpnnlQvyMd-oKtbl03Q0RdkERw3iCFf2...,coutos simiae,False,3,2254123794
8,MaAGkvfuraI5CSUjbyLptqYi-Enw5Cp-rWRCtSKT8JTt8M...,BobaDeathFett,False,4,2254123794
9,pPAANYjxgtvO8YPiSSjT6j9qHRbPfdE6ZNLfT6Ndyxk-pm...,vitors simiae,False,3,2254123794


In [66]:
infos_test = get_match_infos_from_id(2254123794, api_key)

In [70]:
infos_test['info']['gameType']['gameMode']

{'gameCreation': 1619909758000,
 'gameDuration': 1562493,
 'gameId': 2254123794,
 'gameMode': 'CLASSIC',
 'gameName': 'teambuilder-match-2254123794',
 'gameStartTimestamp': 1619909873775,
 'gameType': 'MATCHED_GAME',
 'gameVersion': '11.9.372.2066',
 'mapId': 11,
 'participants': [{'assists': 21,
   'baronKills': 0,
   'bountyLevel': 1,
   'champExperience': 8744,
   'champLevel': 12,
   'championId': 267,
   'championName': 'Nami',
   'championTransform': 0,
   'consumablesPurchased': 7,
   'damageDealtToBuildings': 939,
   'damageDealtToObjectives': 1472,
   'damageDealtToTurrets': 939,
   'damageSelfMitigated': 4125,
   'deaths': 4,
   'detectorWardsPlaced': 4,
   'doubleKills': 0,
   'dragonKills': 0,
   'firstBloodAssist': False,
   'firstBloodKill': False,
   'firstTowerAssist': False,
   'firstTowerKill': False,
   'gameEndedInEarlySurrender': False,
   'gameEndedInSurrender': False,
   'goldEarned': 7128,
   'goldSpent': 6050,
   'individualPosition': 'UTILITY',
   'inhibitorKi